In [50]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [3]:
from symbol import Symbol
s = Symbol()
slist = list(s.symbol_list.index)
edel = pd.read_hdf('edelweiss.h5', 'list')
ede_is_in_iden = edel.isin_identifier
investing_data = pd.read_hdf('investing.h5', 'list')

In [4]:
url = 'https://in.investing.com/instruments/HistoricalDataAjax'
headers= {
    'Accept':'text/plain, */*; q=0.01',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'en-US,en;q=0.8',
    'Connection':'keep-alive',
    'Content-Length':'103',
    'Content-Type':'application/x-www-form-urlencoded',
    'DNT':'1',
    'Host':'www.investing.com',
    'Origin':'https://www.investing.com',
    'Referer':'https://www.investing.com/equities/infosy-tech-historical-data?cid=18217',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest',
    'Cookie': 'adBlockerNewUserDomains=1478707576; optimizelyEndUserId=oeu1478707574692r0.13842377698948205; __qca=P0-745526281-1478707577882; __gads=ID=1d3ab2b8c783a3ad:T=1478707583:S=ALNI_MZYWAu2w292-Zb_CAVRxd4E_k1vWA; editionPostpone=1484144542066; geoC=IN; gtmFired=OK; PHPSESSID=8cr1591k9cp1uljpjp198phv91; StickySession=id.83026372698.670.www.investing.com; _gat=1; _gat_allSitesTracker=1; searchedResults=[{"pairId":18217,"link":"/equities/infosy-tech?cid=18217","symbol":"INFY","name":"Infosys%20Limited","type":"Equity%20-%20NSE","flag":"India"},{"pairId":44675,"link":"/etfs/iq-global-agribusiness-small-cap","symbol":"CROP","name":"IQ%20Global%20Agribusiness%20Small%20Cap","type":"ETF%20-%20NYSE","flag":"USA"},{"pairId":18083,"link":"/equities/crompton-greaves","symbol":"CROM","name":"Crompton%20Greaves%20Ltd.","type":"Equity%20-%20NSE","flag":"India"},{"pairId":17978,"link":"/equities/3m-india","symbol":"TMIN","name":"3M%20India%20Ltd.","type":"Equity%20-%20NSE","flag":"India"}]; SideBlockUser=a%3A2%3A%7Bs%3A10%3A%22stack_size%22%3Ba%3A1%3A%7Bs%3A11%3A%22last_quotes%22%3Bi%3A8%3B%7Ds%3A6%3A%22stacks%22%3Ba%3A1%3A%7Bs%3A11%3A%22last_quotes%22%3Ba%3A7%3A%7Bi%3A0%3Ba%3A3%3A%7Bs%3A7%3A%22pair_ID%22%3Bs%3A5%3A%2217947%22%3Bs%3A10%3A%22pair_title%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22pair_link%22%3Bs%3A24%3A%22%2Findices%2Fnifty-midcap-50%22%3B%7Di%3A1%3Ba%3A3%3A%7Bs%3A7%3A%22pair_ID%22%3Bs%3A5%3A%2217945%22%3Bs%3A10%3A%22pair_title%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22pair_link%22%3Bs%3A20%3A%22%2Findices%2Fs-p-cnx-500%22%3B%7Di%3A2%3Ba%3A3%3A%7Bs%3A7%3A%22pair_ID%22%3Bi%3A18217%3Bs%3A10%3A%22pair_title%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22pair_link%22%3Bs%3A31%3A%22%2Fequities%2Finfosy-tech%3Fcid%3D18217%22%3B%7Di%3A3%3Ba%3A3%3A%7Bs%3A7%3A%22pair_ID%22%3Bs%3A4%3A%226376%22%3Bs%3A10%3A%22pair_title%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22pair_link%22%3Bs%3A22%3A%22%2Fequities%2Fqualcomm-inc%22%3B%7Di%3A4%3Ba%3A3%3A%7Bs%3A7%3A%22pair_ID%22%3Bs%3A4%3A%226401%22%3Bs%3A10%3A%22pair_title%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22pair_link%22%3Bs%3A21%3A%22%2Fequities%2Finfosy-tech%22%3B%7Di%3A5%3Ba%3A3%3A%7Bs%3A7%3A%22pair_ID%22%3Bs%3A5%3A%2218083%22%3Bs%3A10%3A%22pair_title%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22pair_link%22%3Bs%3A26%3A%22%2Fequities%2Fcrompton-greaves%22%3B%7Di%3A6%3Ba%3A3%3A%7Bs%3A7%3A%22pair_ID%22%3Bs%3A5%3A%2244675%22%3Bs%3A10%3A%22pair_title%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22pair_link%22%3Bs%3A38%3A%22%2Fetfs%2Fiq-global-agribusiness-small-cap%22%3B%7D%7D%7D%7D; optimizelySegments=%7B%224225444387%22%3A%22gc%22%2C%224226973206%22%3A%22referral%22%2C%224232593061%22%3A%22false%22%2C%225010352657%22%3A%22none%22%7D; optimizelyBuckets=%7B%7D; billboardCounter_1=0; nyxDorf=MTYxYmQxM3FmMTs1YTsxLTBjN21jYzAsYmYyMw%3D%3D; _ga=GA1.2.1419860609.1478707577; optimizelyPendingLogEvents=%5B%5D'
}
data = {
    'action':'historical_data',
    'curr_id':'18217', 
    'st_date':'01/01/1990',
    'end_date':'01/25/2017',
    'interval_sec':'Daily'
}

In [63]:
url = 'https://www.screener.in/api/company/20MICRONS/consolidated/'
headers= {
    'authority': 'www.screener.in',
    'method': 'GET',
    'path':'api/company/20MICRONS/consolidated/',
    'scheme': 'https',
    'accept': 'application/json',
    'accept-encoding': 'gzip, deflate, sdch, br',
    'accept-language': 'en-US,en;q=0.8',
    'dnt': '1',
    'referer': 'https://www.screener.in/dash/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
}
data = {
    'q': 'i'
}
sess = requests.session()
resp = sess.get(url, headers=headers, verify=False)

In [101]:
cons = json.loads(resp.text)
# pd.DataFrame(cons['number_set']['annual'][0][1].valuess(), index=cons['number_set']['annual'][0][1].keys())
pd.DataFrame.from_dict(cons['number_set']['annual'][0][1], orient='index')
cons['number_set']

{'annual': [['Sales',
   {'2009-03-31': 138.0,
    '2010-03-31': 180.24,
    '2011-03-31': 255.3,
    '2012-03-31': 302.29,
    '2013-03-31': 313.66,
    '2014-03-31': 329.17,
    '2015-03-31': 358.08,
    '2016-03-31': 359.63}],
  ['Expenses',
   {'2009-03-31': 125.4,
    '2010-03-31': 160.19,
    '2011-03-31': 229.17,
    '2012-03-31': 266.18,
    '2013-03-31': 283.51,
    '2014-03-31': 300.18,
    '2015-03-31': 328.17,
    '2016-03-31': 313.39}],
  ['Operating Profit',
   {'2009-03-31': 12.6,
    '2010-03-31': 20.05,
    '2011-03-31': 26.13,
    '2012-03-31': 36.11,
    '2013-03-31': 30.15,
    '2014-03-31': 28.99,
    '2015-03-31': 29.91,
    '2016-03-31': 46.24}],
  ['OPM',
   {'2009-03-31': 9.13,
    '2010-03-31': 11.12,
    '2011-03-31': 10.24,
    '2012-03-31': 11.95,
    '2013-03-31': 9.61,
    '2014-03-31': 8.81,
    '2015-03-31': 8.35,
    '2016-03-31': 12.86}],
  ['Other Income',
   {'2009-03-31': 1.5,
    '2010-03-31': 1.86,
    '2011-03-31': 1.9,
    '2012-03-31': 2.13,
 

In [7]:
sess = requests.session()
# resp = sess.post(url, headers=headers, data=data, verify=False)

In [ ]:
resp.text
soup = BeautifulSoup(resp.text, 'lxml')
temp_data = pd.read_html(str(soup.table))[0]
temp_data

In [8]:
inv_list = investing_data.pair_ID
symbol_data = pd.DataFrame(columns=[
    'symbol', 'date', 'close', 'open', 'high', 'low', 'vol', 'simple_returns'
])

for symbol, inv_id in inv_list.iteritems():
    try:

        data = {
            'action':'historical_data',
            'curr_id': str(inv_id), 
            'st_date':'01/01/2000',
            'end_date':'01/25/2017',
            'interval_sec':'Daily'
        }
        resp = sess.post(url, headers=headers, data=data, verify=False)
        soup = BeautifulSoup(resp.text, 'lxml')
        temp_data = pd.read_html(str(soup.table))[0]
        temp_data.columns = ['date', 'close', 'open', 'high', 'low', 'vol', 'simple_returns']
        temp_data.insert(0, 'symbol', symbol)
        temp_data.loc[:, 'date'] = pd.to_datetime(temp_data.date)
        temp_data = temp_data.sort_values('date')
        symbol_data = symbol_data.append(temp_data)
        print('Succesful for {0}'.format(symbol))
    except Exception as e:
        print('Failed for {0} due to {1}'.format(symbol, e))
        

Succesful for 20microns
Succesful for 3iinfotech
Succesful for 3mindia
Succesful for 8kmiles
Succesful for a2zinfra
Succesful for aartidrugs
Succesful for aartiind
Succesful for aarveeden
Succesful for aban
Succesful for abb
Succesful for abbotindia
Succesful for abfrl
Succesful for abgship
Succesful for abirlanuvo
Succesful for acc
Succesful for accelya
Succesful for ace
Succesful for adanient
Succesful for adaniports
Succesful for adanipower
Succesful for adanitrans
Succesful for adffoods
Succesful for adhunik
Succesful for adhunikind
Succesful for adi
Succesful for adlabs
Succesful for adorweld
Succesful for advanihotr
Succesful for advenzymes
Succesful for aegischem
Succesful for afl
Succesful for agarind
Succesful for agcnet
Succesful for agritech
Succesful for ahleast
Succesful for ahlucont
Succesful for ahlwest
Succesful for aiaeng
Succesful for aifl
Succesful for ajantpharm
Succesful for ajmera
Succesful for akshoptfbr
Succesful for akzoindia
Succesful for alankit
Succesful for

In [ ]:
 data = {
    'action':'historical_data',
    'curr_id': str(18217),
    'st_date':'01/01/2000',
    'end_date':'01/01/2016',
    'interval_sec':'Daily'
}
resp = sess.post(url, headers=headers, data=data, verify=False)
soup = BeautifulSoup(resp.text, 'lxml')
temp_data = pd.read_html(str(soup.table))[0]
temp_data.columns = ['date', 'close', 'open', 'high', 'low', 'vol', 'simple_returns']
temp_data.loc[:, 'date'] = pd.to_datetime(temp_data.date)

In [ ]:
temp_data.groupby(temp_data.date.dt.year).date.count()

In [ ]:
temp_data.groupby(temp_data.date.dt.year).date.count()

In [ ]:
temp_data.groupby(temp_data.date.dt.year).date.count()

In [ ]:
symbol_data.groupby('symbol').count()

In [ ]:
search_url = 'https://www.investing.com/search/service/search'

search_headers= {
    'Accept':'text/plain, */*; q=0.01',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'en-US,en;q=0.8',
    'Connection':'keep-alive',
    'Content-Length':'103',
    'Content-Type':'application/x-www-form-urlencoded',
    'DNT':'1',
    'Host':'www.investing.com',
    'Origin':'https://www.investing.com',
    'Referer':'https://www.investing.com/equities/infosy-tech-historical-data?cid=18217',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest'
    
}

search_data = {
    'search_text':'3mindia',
    'term':'3mindia',
    'country_id':'0',
    'tab_id':'All'
}


In [ ]:
investing_data = pd.DataFrame(index=slist, columns=[
        'pair_ID', 'link', 'trans_name', 'aql_pre_link', 
        'exchange_popular_symbol', 'override_country_ID', 'name', 
        'exchange_name_short', 'country_ID', 'tab_ID', 'symbol', 
        'aql_link', 'pair_type', 'popularity_rank', 'flag', 'pair_type_label'
    ])
for symbol, isin in ede_is_in_iden.iteritems():
    try:
        search_data = {
            'search_text': isin,
            'term': isin,
            'country_id':'14',
            'tab_id':'All',
            'exchange_popular_symbol':'NS'
        }
        search_response = sess.post(search_url, headers = search_headers, data=search_data)
        search_json = json.loads(search_response.text)['All'][0]
        symbol_series = pd.Series(search_json)
        investing_data.loc[symbol] = symbol_series
        print('Successful for {0}'.format(symbol))
    except Exception as e:
        print('Failed for {0} due to {1}'.format(symbol, e))
investing_data = investing_data.dropna(how='all').convert_objects(convert_numeric=True)


In [ ]:
investing_data.to_hdf('investing.h5', 'list')
symbol_data.to_hdf('investing.h5', 'data')

In [10]:
symbol_data = symbol_data.sort_values(['symbol', 'date'])

In [11]:
close = symbol_data.pivot(index='date', columns='symbol', values='close')

In [12]:
close[close.columns[close.ix['2016'].count() == 247]]['2016']

symbol,20microns,3iinfotech,3mindia,8kmiles,a2zinfra,aartidrugs,aartiind,aban,abb,abbotindia,...,zandurealt,zeel,zeelearn,zeemedia,zensartech,zentec,zicom,zuari,zuariglob,zyduswell
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,35.20,5.15,11321.95,832.33,27.60,595.30,525.10,226.40,1116.45,5895.95,...,1535.45,432.90,39.30,23.85,1086.60,124.50,134.10,159.50,107.50,863.35
2016-01-04,35.90,5.20,10935.80,893.23,26.40,588.85,520.80,229.85,1120.15,5930.90,...,1534.15,421.95,37.85,23.25,1065.75,121.05,130.25,157.70,113.35,858.10
2016-01-05,35.55,5.40,10784.35,899.96,27.10,590.10,523.60,230.30,1094.55,5878.10,...,1499.60,425.05,38.25,23.25,1065.80,125.15,131.30,160.50,133.85,864.70
2016-01-06,35.75,5.50,10644.60,914.70,27.35,587.40,516.30,225.90,1077.60,5783.60,...,1520.35,423.15,37.85,23.05,1064.85,125.90,129.55,168.25,125.15,863.80
2016-01-07,34.10,5.40,10673.75,876.36,26.35,562.70,510.90,213.15,1053.45,5656.15,...,1482.75,415.40,36.60,21.85,1019.80,121.15,123.40,162.50,120.90,850.55
2016-01-08,35.05,5.70,10589.60,891.43,27.15,565.60,507.50,215.60,1055.55,5693.85,...,1470.60,411.70,36.65,21.85,1076.95,124.05,125.10,173.80,122.25,837.95
2016-01-11,36.55,5.65,10653.95,883.76,26.65,560.00,508.80,213.05,1006.85,5646.35,...,1605.95,409.60,35.65,21.20,1067.45,122.45,123.20,177.05,117.70,845.65
2016-01-12,36.20,5.75,10657.05,804.58,27.40,560.50,514.95,208.35,1026.35,5537.10,...,1549.85,407.05,36.15,21.00,1056.55,120.35,121.45,176.40,116.40,845.15
2016-01-13,33.50,5.30,10514.30,931.39,26.15,557.90,505.10,201.90,1019.85,5443.80,...,1497.30,414.25,34.50,20.60,1016.00,108.75,116.85,169.40,112.75,824.50


In [16]:
from helpers import num_missing

In [ ]:
close.count().idxmax()

In [ ]:
close['abirlanuvo'].dropna()

In [43]:
syms = close.columns[close.apply(num_missing, start='valid_index') <= 6]

In [44]:
'infy' in syms, len(syms)

(True, 834)

In [45]:
len(symbol_data)

2048167

In [49]:
s.fetch_historical_data('infy')

Recieved 5253 records from NSE for infy from 1996-01-01 to 2017-01-25


,date,symbol,prev_close,open,high,low,last,close,vwap,volume,turnover,pct_deliverble,simple_returns,log_returns,daily_volatility
0,1996-01-01,infy,408.00,407.00,407.90,405.00,NaN,407.90,406.48,400.0,1.630000e+05,NaN,-0.000245,-0.000245,0.000060
1,1996-01-02,infy,407.90,407.00,409.00,406.25,NaN,406.25,407.58,400.0,1.630000e+05,NaN,-0.004045,-0.004053,0.000993
2,1996-01-03,infy,406.25,409.00,409.00,409.00,NaN,409.00,409.00,200.0,8.200000e+04,NaN,0.006769,0.006746,0.001653
3,1996-01-04,infy,409.00,405.00,407.00,405.00,NaN,406.30,405.65,600.0,2.430000e+05,NaN,-0.006601,-0.006623,0.001622
4,1996-01-05,infy,406.30,401.50,401.50,401.50,NaN,401.50,401.50,100.0,4.000000e+04,NaN,-0.011814,-0.011884,0.002911
5,1996-01-08,infy,401.50,401.50,405.00,402.00,NaN,404.00,404.14,700.0,2.830000e+05,NaN,0.006227,0.006207,0.001520
6,1996-01-09,infy,404.00,404.00,400.00,395.00,NaN,399.50,399.53,1700.0,6.790000e+05,NaN,-0.011139,-0.011201,0.002744
7,1996-01-10,infy,399.50,399.50,397.00,395.00,NaN,396.00,395.29,700.0,2.770000e+05,NaN,-0.008761,-0.008800,0.002155
8,1996-01-11,infy,396.00,399.00,405.00,396.00,NaN,405.00,400.44,1600.0,6.410000e+05,NaN,0.022727,0.022473,0.005505
9,1996-01-12,infy,405.00,403.00,403.00,400.00,NaN,402.75,401.35,700.0,2.810000e+05,NaN,-0.005556,-0.005571,0.001365
